In [35]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import os.path
import psycopg2

import configparser

from PIL import Image


In [2]:
# Path to the sql script used for getting images and details from database.
SELECT_IMAGE_SCRIPT_PATH = "database/select_aircraft_images.sql"

# Used as a performance argument.
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Batch size for data set to process.
BATCH_SIZE = 32

# The file used for caching the dataset.
DATASET_CACHE_FILE = "dataset_cache"

IMG_HEIGHT = 128
IMG_WIDTH = 128

In [3]:
class DBConfig:
    def __init__(self, filename, host, port, db_name, user, pwd):
        self.filename = filename
        self.host = host
        self.port = port
        self.db_name = db_name
        self.user = user
        self.pwd = pwd

In [4]:
def load_config(filename = 'database.ini', section = 'aircraft_postgres_db'):
    parser = configparser.ConfigParser()
    parser.read(filename)
    cfg = parser[section]
    return DBConfig(filename, cfg['Host'], int(cfg['Port']), cfg['Db_name'], cfg['User'], cfg['Pass'])

In [5]:
def load_sql(filename):
    with open(filename, "r") as sql_script_file:
        return sql_script_file.read()

In [36]:
def ready_image(path):
    # Path must be transformed into the actual image.
    
#     # Variant name is the variantid of the image.
#     variant_id = image_details[1]

#     path = image_details[0]
#     label = image_details[1]
    
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    
    print(img)
    
    return img

In [7]:
def _prepare_image_dataset(image_paths, labels):
    image_paths_dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    
    print(image_paths_dataset)
    
    ds = image_paths_dataset.map(ready_image)
    
    print(ds)
    
#     ds = ds.cache(DATASET_CACHE_FILE)
    
    
    # ds = ds.repeat()
    
    
#     ds = ds.batch(BATCH_SIZE)
#     ds = ds.prefetch(buffer_size=AUTOTUNE)
    
    return ds
    

In [8]:
def train_on_images(image_paths):
    dataset = tf.data.Dataset.from_tensor_slices(image_paths);
    for (path, variant_name) in image_paths:
        img = read_parse_image(path)

In [9]:
# Taken from https://www.tensorflow.org/tutorials/load_data/images (09/06/2020)
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(label_batch[n].title())
      plt.axis('off')

In [10]:
db_cfg = load_config()

In [11]:
conn = psycopg2.connect(dbname=db_cfg.db_name, user=db_cfg.user, password=db_cfg.pwd, host=db_cfg.host, port=db_cfg.port)

In [20]:
cur = conn.cursor()
cur.execute('SELECT version()')
print("Connected to database, version: " + str(cur.fetchone()))

Connected to database, version: ('PostgreSQL 12.3, compiled by Visual C++ build 1914, 64-bit',)


In [21]:
select_image_sql = load_sql(SELECT_IMAGE_SCRIPT_PATH)

In [22]:
cur.execute(select_image_sql)
training_data = cur.fetchmany(size=100)

In [45]:
def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3) #color images

    #convert unit8 tensor to floats in the [0,1]range
    img = tf.image.convert_image_dtype(img, tf.float32)
    
    #resize the image into 224*224
    return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT]) 

In [53]:
def process_img_path(path):
    img = tf.io.read_file(path)
    img = decode_img(img)
    return img

In [54]:
def process_img_data(img_data):
    path = img_data[0]
    label = img_data[1]
    img = process_img_path(path)
    return img, label

In [55]:
# print(list(training_data))

In [61]:
# Split the training image paths from the labels.
image_paths, labels = zip(*training_data)

image_paths = list(image_paths)
labels = list(labels)

image_data = list(map(process_img_path, image_paths))

# ds = tf.data.Dataset.from_tensor_slices(training_data)

# labeled_ds = ds.map(process_img_data, num_parallel_calls=AUTOTUNE)

# print(image_paths)

# image_paths = list(map(lambda x : (x[0], str(x[1])), image_paths))

image_count = len(labels)

print("Loaded {} training images".format(image_count))

# print(image_data)

labeled_ds = tf.data.Dataset.from_tensor_slices((image_data, labels))

for image, label in labeled_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Loaded 100 training images
Image shape:  (128, 128, 3)
Label:  408


In [62]:
# images = tf.data.Dataset.from_tensor_slices(image_paths)

# images = prepare_image_dataset(image_paths, labels)


# for x in image_paths:
#     image_data.append(ready_image(x))

# print(image_data)
    
# img_batch, label_batch = next(iter(image_dataset))

# show_batch(img_batch.numpy(), label_batch.numpy())


# Data is now loaded and ready, time to actually handle the model

In [63]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

In [80]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

In [81]:
# model.compile(optimizer='adam',
#               loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#               metrics=['accuracy'])

In [82]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [88]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 128, 128, 16)      448       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 16384)            

In [95]:
# image_data = image_data.batch(10)

batch_labeled_ds = labeled_ds.batch(10)
batch_labeled_ds = batch_labeled_ds.repeat()

In [96]:
batch_size = 10
epochs = 3

history = model.fit(batch_labeled_ds, epochs=2, steps_per_epoch=10)

Epoch 1/2
10/10 [==============================] - 1s 60ms/step - loss: 5.1910e-05 - accuracy: 0.0000e+00
Epoch 2/2
10/10 [==============================] - 1s 61ms/step - loss: 5.1910e-05 - accuracy: 0.0000e+00
